In [158]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

In [159]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.0
torch.manual_seed(1337)

In [160]:
stoi = {
    ' ': 30,
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
    'h': 8,
    'i': 9,
    'j': 10,
    'k': 11,
    'l': 12,
    'm': 13,
    'n': 14,
    'o': 15,
    'p': 16,
    'q': 17,
    'r': 18,
    's': 19,
    't': 20,
    'u': 21,
    'v': 22,
    'w': 23,
    'x': 24,
    'y': 25,
    'z': 26,
    '#': 27,
    '$': 28,
    '@': 29,
    '%': 0
}
# '# = is to'
# '$ = as'
# '@ = end token'
# '% = padding at the start to make it block size length'
vocab_size = len(stoi)

itos = {v: k for k, v in stoi.items()}

# def tokenize(text):
#     return [CHAR_TO_TOKEN[c] for c in text]

# def detokenize(tokens):
#     return ''.join([TOKEN_TO_CHAR[t] for t in tokens])

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [161]:
word_pairs = []
with open('nouns.csv', 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) >= 2:
            singular = parts[0].strip()
            plural = parts[1].strip()

            if all(c in stoi for c in singular) and all(c in stoi for c in plural) and len(singular) + len(plural) <= 13:
                word_pairs.append((singular, plural))

print(word_pairs[100])

('absurd', 'absurds')


In [162]:
n = int(0.9*len(word_pairs)) # first 90% will be train, rest val
train_data = word_pairs[:n]
val_data = word_pairs[n:]

In [163]:
def get_context_examples(split):
    '''
    Data is of the form:
    %%%% singular $ plural # singular $ part of a plural

    We pad with %%% to keep the size correct

    For example:
    x = %%%%%%%fox$foxes#dog$do
    y = %%%%%%fox$foxes#dog$dog
    '''

    data = train_data if split == 'train' else val_data
    n = len(data)
    s1, p1 = data[np.random.randint(n)]
    s2, p2 = data[np.random.randint(n)]
    window = np.random.randint(len(p2)+1)
    p2 += '@'
    x = s1 + '#' + p1 + '$' + s2 + '#' + p2[:window]
    y = s1[1:] + '#' + p1 + '$' + s2 + '#' + p2[:window + 1]
    # x = (s1 + '#' + p1 + '$' + s2 + '#' + p2[:window]).rjust(block_size, '%')
    # y = (s1 + '#' + p1 + '$' + s2 + '#' + p2[:window + 1]).rjust(block_size, '%')
    x = torch.tensor(encode(x))
    y = torch.tensor(encode(y))
    return x, y

def get_batch(split):
    data = train_data if split == 'train' else val_data
    x = torch.zeros((batch_size, block_size), dtype=int)
    y = torch.zeros((batch_size, block_size), dtype=int)
    mask = torch.zeros((batch_size, block_size), dtype=torch.bool)


    for i in range(batch_size):
        xi, yi = get_context_examples(split)
        x[i, :len(xi)] = xi  # Assign values up to the original length
        y[i, :len(yi)] = yi  # The rest will be padded with 0s
        mask[i, :len(xi)] = True  # Set True for actual data tokens
        #x[i] = xi
        #y[i] = yi

    x, y, mask = x.to(device), y.to(device), mask.to(device)
    return x, y, mask

# batch = get_batch('train')
# print(batch[0].shape)
# print(batch[1].shape)

In [164]:
example = get_context_examples('train')
print(decode(example[0].tolist()))
print(decode(example[1].tolist()))

harle#harles$infare#infare
arle#harles$infare#infares


In [165]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y, mask = get_batch(split)
            logits, loss = model(X, Y, mask)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        if mask is not None:
            # Apply the mask before the softmax
            wei = wei.masked_fill(~mask[:, None, :], float('-inf'))

        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        #out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = torch.cat([h(x, mask=mask) for h in self.heads], dim=-1)  # Pass mask

        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, mask=None):
        x = x + self.sa(self.ln1(x), mask=mask)
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, mask=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        for block in self.blocks:
          x = block(x, mask=mask)

        #x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)


            # # Create a mask for padding tokens
            # mask = (targets != stoi['%']).float() # 1 for non-padding, 0 for padding

            # # Apply the mask to the loss calculation
            # loss = F.cross_entropy(logits, targets, reduction='none') * mask
            # loss = loss.sum() / mask.sum()  # Average over non-padding tokens

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    def evaluate(self, num_to_evaluate):
        data = val_data
        n = len(data)

        correct = 0
        total = 0
        for _ in range(num_to_evaluate):
            s1, p1 = data[np.random.randint(n)]
            s2, p2 = data[np.random.randint(n)]
            p2 += '@'

            #idx = (s1 + '#' + p1 + '$' + s2 + '#').rjust(block_size, '%')
            idx = s1 + '#' + p1 + '$' + s2 + '#'
            context = idx[:]
            idx = torch.tensor(encode(idx)).to(device).unsqueeze(0)
            #print(idx)
            for i in range(len(p2)):
                # crop idx to the last block_size tokens
                idx_cond = idx[:, -block_size:]
                # get the predictions
                logits, loss = self(idx_cond)
                # focus only on the last time step
                logits = logits[:, -1, :] # becomes (B, C)
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (B, C)
                #print(f"Probs = {probs}")
                # sample from the distribution
                idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
                # append sampled index to the running sequence
                idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
                #print(idx)

            predicted = decode(idx[0].tolist())[-len(p2):]

            print(f"Context: {context}, Prediction: {predicted}, Correct: {p2}")
            if predicted == p2:
                correct += 1
            total += 1

        print(f"Accuracy: {correct}/{total}")

In [166]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb, mask = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb, mask)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
print(m.evaluate(20))

10.713631 M parameters
step 0: train loss 3.6182, val loss 3.6501
step 500: train loss 0.2054, val loss 0.2504
step 1000: train loss 0.1904, val loss 0.2368
step 1500: train loss 0.1846, val loss 0.2353
step 2000: train loss 0.1792, val loss 0.2450
step 2500: train loss 0.1740, val loss 0.2458
step 3000: train loss 0.1673, val loss 0.2506
step 3500: train loss 0.1619, val loss 0.2518
step 4000: train loss 0.1572, val loss 0.2591
step 4500: train loss 0.1526, val loss 0.2671
step 4999: train loss 0.1501, val loss 0.2675
Context: zaa#zaas$wazzer#, Prediction: wazzers@, Correct: wazzers@
Context: zaim#zaims$yowler#, Prediction: yowlers@, Correct: yowlers@
Context: vamper#vampers$zine#, Prediction: zines@, Correct: zines@
Context: wheal#wheals$urodid#, Prediction: urodids@, Correct: urodids@
Context: witigo#witigos$voe#, Prediction: %%%%%, Correct: voes@
Context: visage#visages$upshot#, Prediction: upshots@, Correct: upshots@
Context: wadge#wadges$yamboo#, Prediction: yamboos@, Correct: ya

In [168]:
torch.save(model.state_dict(), 'model_weights.pth')


In [170]:
model = GPTLanguageModel()  # Create a new model with the same architecture
model.to(device) # move to device
model.load_state_dict(torch.load('model_weights.pth'))
print(m.evaluate(1))

Context: wold#wolds$waving#, Prediction: wavings@, Correct: wavings@
Accuracy: 0/1
None
